In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

c:\Users\user\Miniconda3\envs\rec\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Preprocessing and Dataset

In [2]:
class PreProcess():
    def __init__(self, dir_path, file_name, *, sep="::", str_cols=['user', 'item', 'rating', 'timestamp'],
                ratio=(7, 1, 2), drop_num=9, drop_rating=3, use_cache=False):
        """
        논문에 따라 pre-processing하는 코드.
        Args:
            dir_path    : Dataset directory path
            file_name   : Interaction 파일 이름
            sep         : Seperator
            str_cols    : Interacton 파일에서 각 columns의 이름.
            ratio       : Train, valid, test 비율
            drop_num    : Unactive한 user 기준. 해당 수보다 interaction의 수가 큰 user만 사용.
            drop_rating : User가 선호하는 item을 만족하는 기준. 해당 rating보다 큰 것을 선호한다고 판단.
            user_cache  : Pre-processing을 된 파일 사용.
        """

        self.str_user, self.str_item, self.str_rating, self.str_time = str_cols
        clean_df_path = os.path.join(dir_path, 'clean_df.csv')
        clean_train_path = os.path.join(dir_path, 'clean_train.csv')
        clean_val_path = os.path.join(dir_path, 'clean_val.csv')
        clean_test_path = os.path.join(dir_path, 'clean_test.csv')

        if use_cache and \
            os.path.exists(clean_df_path) and \
            os.path.exists(clean_train_path) and \
            os.path.exists(clean_val_path) and \
            os.path.exists(clean_test_path):
            # Pre-processing을 사용하고, 해당 파일들이 있으면 사용.
            self.df_clean = pd.read_csv(clean_df_path)
            self.df_train = pd.read_csv(clean_train_path)
            self.df_val = pd.read_csv(clean_val_path)
            self.df_test = pd.read_csv(clean_test_path)
        else:
            file_path = os.path.join(dir_path, file_name)
            self.df = pd.read_csv(file_path, sep=sep, engine ='python', names=str_cols)

            self.le_user = LabelEncoder() # user id encoder: 0, 3, 5, ... 처럼 연속되지 않을 수도 있기 때문에 연속적으로 만듦.
            self.le_item = LabelEncoder() # item id encoder
            self.df_clean = self.clean_and_sort(drop_num, drop_rating) # Unactive user를 drop하고 시간 순서대로 sorting
            self.df_train, self.df_val, self.df_test = self.split_group_by_user(self.df_clean, ratio) # User마다 ratio만큼 train, valid, test data 만듦.

            self.df_clean.to_csv(clean_df_path, index=False)
            self.df_train.to_csv(clean_train_path, index=False)
            self.df_val.to_csv(clean_val_path, index=False)
            self.df_test.to_csv(clean_test_path, index=False)

        self.num_users = self.df_clean[self.str_user].nunique() # User의 수: sparse matrix를 만들기 위해 필요
        self.num_items = self.df_clean[self.str_item].nunique() # Item의 수: sparse matrix를 만들기 위해 필요

        # Train, valid, test를 sparse matrix로 만듦.
        self.sp_train = self.make_csr_matrix(self.df_train)
        self.sp_val = self.make_csr_matrix(self.df_val)
        self.sp_test = self.make_csr_matrix(self.df_test)

    def clean_and_sort(self, drop_num, drop_rating):
        df_clean = self.drop_unreliable(self.df, drop_rating)
        df_clean = self.drop_unactive(df_clean, drop_num)
        df_clean[self.str_rating] = 1.0 # rating 값을 사용하지 않고, implicit으로 사용.
        df_sorted = df_clean.sort_values([self.str_user, self.str_time])
        df_sorted[self.str_user] = self.le_user.fit_transform(df_sorted[self.str_user])
        df_sorted[self.str_item] = self.le_user.fit_transform(df_sorted[self.str_item])
        return df_sorted
    
    def drop_unreliable(self, df, drop_rating):
        df_clean = df[df[self.str_rating] > drop_rating]
        return df_clean

    def drop_unactive(self, df, drop_num):
        group_user_size = df.groupby(self.str_user).size()
        clean_user = group_user_size[group_user_size > drop_num].index
        df_clean = df[df[self.str_user].isin(clean_user)]
        return df_clean
    
    def split_group_by_user(self, df, ratio):
        group_users = df.groupby(self.str_user)
        train = pd.DataFrame()
        val = pd.DataFrame()
        test = pd.DataFrame()

        sum_ratio = sum(ratio)
        test_ratio = round(ratio[2] / sum_ratio, 3)
        sum_ratio -= ratio[2]
        val_ratio = round(ratio[1] / sum_ratio, 3)

        for _, df_user in group_users:
            try:
                sum_ratio = sum(ratio)
                test_ratio = round(ratio[2] / sum_ratio, 3)
                df_train_val, df_test = train_test_split(df_user, test_size=test_ratio, shuffle=False)
                df_train, df_val = train_test_split(df_train_val, test_size=val_ratio, shuffle=False)
                
                train = pd.concat([train, df_train], ignore_index=True)
                val = pd.concat([val, df_val], ignore_index=True)
                test = pd.concat([test, df_test], ignore_index=True)
            except:
                print(df_user)
        
        return train, val, test
    
    def make_csr_matrix(self, df):
        data = df[self.str_rating].values
        row = df[self.str_user].values
        col = df[self.str_item].values
        return sp.csr_matrix((data, (row, col)), dtype='float32', shape=(self.num_users, self.num_items))

In [3]:
class DataDiffusion(Dataset):
    def __init__(self, dataset):
        self.data = dataset

    def __getitem__(self, index): 
        item = self.data[index]
        return item
        
    def __len__(self):
        return len(self.data)

### 2. Diffusion and Model

#### 2.1. Diffusion

In [4]:
class Diffusion():
    def __init__(self, steps=100, beta_start=1e-4, beta_end=0.02, device='cuda',\
            noise_scale=0.1, num_for_expectation=10):
        """
        Forward diffusion 또는 주어진 model로 reverse diffusion한다.
        Args:
            steps               : reverse할 개수
            beta_start          : Beta 시작 값, DDPM 논문에 적힌 1e-4 사용.
            beta_end            : Beta 마지막 값, DDPM 논문에 적힌 0.02 사용. 
            noise_scale         : Beta를 생성할 때, noise의 정도를 조정하기 위해 사용.
                                : 논문 3.4 personalized recommendation 1)의 마지막 문장 참고.
            num_for_expectation : Importance sampling에 사용되는, expectation을 구하기 위해 필요한 loss의 수
        """
        self.steps = steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.device = device
        self.noise_scale = noise_scale

        self.beta = torch.tensor(self.get_betas(), dtype=torch.float32).to(device)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        self.alpha_bar_prev = torch.cat([torch.tensor([1.0]).to(device), self.alpha_bar[:-1]]).to(device) # 제일 처음 원소는 어차피 안 쓰임.
        self.sqrt_alpha_bar = torch.sqrt(self.alpha_bar)
        self.sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar)

        # 논문 수식 8
        self.posterior_mean_coef1 = torch.sqrt(self.alpha) * (1 - self.alpha_bar_prev) / (1.0 - self.alpha_bar) # x_t 앞의 계수
        self.posterior_mean_coef2 = torch.sqrt(self.alpha_bar_prev) * self.beta / (1.0 - self.alpha_bar) # x_0 앞의 계수
        # 아래 부분은 posterior variance를 사용하는 값인 것 같다.
        # 원래 DDPM에서는 beta를 사용했지만, 학습할 수도 있고, 아래처럼 다양한 것을 사용할 수 있다.
        # 본 논문에서는 따로 언급은 없지만 log_var_clipped를 사용했다.
        self.posterior_variance = self.beta * (1.0 - self.alpha_bar_prev) / (1.0 - self.alpha_bar)
        self.posterior_log_variance_clipped = torch.log(
            torch.cat([self.posterior_variance[1].unsqueeze(0), self.posterior_variance[1:]])
        )

        # Step을 sampling 하는 방법 중 importance sampling에 필요한 variable.
        # Importance sampling은 DDPM을 향상 시키는 technique 중 하나.
        # Importance sampling은 각 step마다 optimization의 어려움 정도가 다르다는 것을 가정한다.
        # 그래서 Loss가 큰 step에 대한 학습을 강조하기 위해 importance sampling을 고려한다.
        # 간단히 말하면 loss가 큰 step에 대해 sampling 확률을 높인다.
        self.num_for_expectation = num_for_expectation # Monte Calro를 사용하기 위한 개수.
        self.Lt_history = torch.zeros(steps, num_for_expectation, dtype=torch.float32).to(device)
        self.Lt_count = torch.zeros(steps, dtype=int).to(device)

    def get_betas(self, max_beta=0.999):
        # DDPM에서는 beta가 linear하게 증가하도록 하고 있는데,
        # 본 논문 eq 4는 alpha_bar가 linear하도록 beta를 설정하고 있다.
        # 풀어서 써보면 alpha_bar = 1 - np.linspace(...)의 값을 가지게 된다.

        start = self.noise_scale * self.beta_start
        end = self.noise_scale * self.beta_end
        alpha_bar = 1 - np.linspace(start, end, self.steps) # 1 - \beta
        betas = []
        betas.append(1 - alpha_bar[0])
        for i in range(1, self.steps):
            betas.append(min(1 - alpha_bar[i] / alpha_bar[i - 1], max_beta))
        return np.array(betas)

    def sample_steps(self, batch_size, method='uniform', uniform_prob=0.001):
        if method == 'importance':
            if not (self.Lt_count == self.num_for_expectation).all():
                # 모든 steps에 대한 loss가 num_for_expectation만큼 없으면 uniform 방식으로 sampling
                return self.sample_steps(batch_size, method='uniform')

            # 수식 14에 따라 sampling한다.
            Lt_sqrt = torch.sqrt(torch.mean(self.Lt_history ** 2, axis=-1)) 
            # 10개 loss의 제곱에 대한 평균의 루트값.
            # Lt_sqrt shape: (steps,)
            
            pt_prob = Lt_sqrt / torch.sum(Lt_sqrt)
            pt_prob *= 1 - uniform_prob 
            pt_prob += uniform_prob / len(pt_prob)
            # Loss의 크기에 따라 sampling 확률을 다르게 준다.
            # 어느 정도 uniform_prob 만큼 sampling 되는 것을 보장.

            step = torch.multinomial(input=pt_prob, num_samples=batch_size, replacement=True) 
            # 중복 sampling
            pt = pt_prob.gather(dim=0, index=step) * len(pt_prob)
            # 각 step의 확률 값을 가져온다.
            # 수식 14에 따라 training에서 Lt / pt를 하기 위함.

        elif method == 'uniform':
            steps = torch.randint(low=1, high=self.steps, size=(batch_size,)).long()
            pt = torch.ones_like(steps).float()
            # loss의 평균 값을 구하기 때문에 len(pt)로 안 나눠줘도 된다.
        
        else: raise ValueError
        
        return steps.to(self.device), pt.to(self.device)

    def get_noised_interaction(self, x_0, t):
        """
        Noise를 추가한, 각 item에 대한 소비할 확률 값을 구하는 함수.
        논문 수식 3 참고.
        Args:
            x_0 : Training dataset으로 만들어진 user interactions   (batch_size, num_items)
            t   : noise step                                       (batch_size, )
        """
        sqrt_alpha_bar = self.sqrt_alpha_bar[t][:, None]
        mean_ = sqrt_alpha_bar * x_0
        std_ = self.sqrt_one_minus_alpha_bar[t][:, None]
        noise = torch.randn_like(x_0)
        # 각 item마다 줄 noise를 sampling한다. -> reparameter trick에서 사용됨.

        noised_interaction = mean_ + std_ * noise # reparmeter
        return noised_interaction, noise


    def sample_new_interaction(self, model, x_0, steps: int, sampling_noise=False):
        """
        x_0부터 정해진 steps만큼 noise를 주고, 
        학습된 model을 가지고 reverse diffusion으로 추천을 생성한다.
        Args:
            model           : 학습된 model
            x_0             : 초기 users의 interactions, shape: (batch_size, num_items), shape (batch_size, num_items)
            steps           : Forward를 할 step, x_T까지 forward하지 않는다. 이유는 논문 3.3 참고.
            sampling_noise  : 추천을 생성할 때, noise 추가 유무. False이면 variance 없이 mean 값만 사용.
        """

        if steps == 0: 
            # noise를 전혀 추가하지 않고, reverse를 T번 진행
            # 기존의 user의 interaction이 noise하다고 가정.
            x_T = x_0
        else:
            T = torch.tensor([steps - 1] * x_0.shape[0]).to(x_0.device)
            x_T = self.get_noised_interaction(x_0, T)

        reverse_t = list(range(self.steps))[::-1]
        
        x_t = x_T
        if self.noise_scale == 0:
            # Denoise 과정이 없다.
            # 즉, forward가 없다. 이러면 각 reverse는 x_t -> x_t를 복원하는 것이다.
            # 결국 x_t -> x_t를 하는 AutoEndocer를 여러 개 쌓은 것과 같다.
            for t_idx in reverse_t:
                t = torch.tensor([t_idx] * x_t.shape[0]).to(x_0.device) # Shape: (batch_size, )
                x_t = model(x_t, t)
        else:
            # Denosing 과정이 있다.
            # x_t와 t가 주어졌을 때, p(x_{t-1}|x_t)의 평균과 분산 값을 구한다.
            # 이를 통해 reparameterzation trick으로 x_{t-1}을 얻는다.
            # 우리는 p(x_{t-1}|x_t)를 모른다. 여러 수식을 유도하면, p(x_{t-1}|x_t)의 likelihodd는 
            # q(x_{t-1}|x_t, x_0)와 유사한 분포를 가지면 커진다.
            # 이때, x_t는 알아도 x_0를 모른다. 그래서 학습된 model을 통해 x_0를 예측한 값을 사용한다.
            for t_idx in reverse_t:
                t = torch.tensor([t_idx] * x_t.shape[0]).to(x_0.device)
                x_0_hat = model(x_t, t)
                mean_hat = self.posterior_mean_coef1[t][:, None] * x_t + self.posterior_mean_coef2[t][:, None] * x_0_hat
                if sampling_noise:
                    # 추천을 생성할 때 uncertainty 추가. 즉, variance 사용
                    variance = self.posterior_log_variance_clipped[t][:, None]
                    if t_idx > 1: noise = torch.randn_like(x_t)
                    else: noise = torch.zeros_like(x_t) # t == 0일 때 noise를 주지 않는다.
                    x_t = mean_hat + torch.exp(0.5 * variance) * noise
                else:
                    # 추천을 생성할 때 variance를 사용하지 않음.
                    # 따라서 평균값만 사용.
                    x_t = mean_hat
        return x_t

#### 2.2. Model

In [5]:
class PosEmb(nn.Module):
    def __init__(self, pos_dim):
        super(PosEmb, self).__init__()
        """
        Sinusoidal timestep positional encoding.
        Args
            pos_dim : embedding dimension
        """
        self.pos_dim = pos_dim
        self.time_mlp = nn.Linear(pos_dim, pos_dim)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.normal_(m.bias.data, mean=0.0, std=0.001)

    def forward(self, t, max_period=10000):
        t = t.unsqueeze(-1).type(torch.float)
        half_dim = self.pos_dim // 2
        w_k = 1.0 / (
            max_period
            ** (torch.arange(0, half_dim, 1, device=t.device).float() / (half_dim-1))
        )

        half_emb = t.repeat(1, half_dim)
        pos_sin = torch.sin(half_emb * w_k)
        pos_cos = torch.cos(half_emb * w_k)
        pos_enc = torch.cat([pos_sin, pos_cos], dim=-1)

        emb = self.time_mlp(pos_enc)
        return emb


class ReverseDiffusion(nn.Module):
    def __init__(self, in_dims, out_dims, step_dim=10, norm=False, droupout=0.5):
        super(ReverseDiffusion, self).__init__()
        """
        AutoEncoder 구조를 활용한다.
        Args
            in_dims     : [num_items, ...]
            out_dims    : [..., num_itmes]
            step_dim    : timestep positional embedding dim.
        """

        self.norm = norm
        self.pos_enc_layer = PosEmb(step_dim)
        self.encoder = nn.ModuleList([])
        self.decoder = nn.ModuleList([])

        in_dims_w_step = [in_dims[0] + step_dim] + in_dims[1:]
        for d_in, d_out in zip(in_dims_w_step[:-1], in_dims_w_step[1:]):
            self.encoder.append(nn.Linear(d_in, d_out))
            self.encoder.append(nn.Tanh())
        for d_in, d_out in zip(out_dims[:-1], out_dims[1:]):
            self.decoder.append(nn.Linear(d_in, d_out))

        self.drop = nn.Dropout(droupout)
        self.init_weights()

    def init_weights(self,):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.normal_(m.bias.data, mean=0.0, std=0.001)

    def forward(self, x, timesteps):
        time_emb = self.pos_enc_layer(timesteps)
        if self.norm: x = F.normalize(x)
        x = self.drop(x)
        h = torch.cat([x, time_emb], dim=-1)
        for idx, layer in enumerate(self.encoder):
            h = layer(h)
        for idx, layer in enumerate(self.decoder):
            h = layer(h)
            if idx != len(self.decoder) - 1:
                h = torch.tanh(h)

        return h

### 3. Training

#### 3.1. utils

In [6]:
def SNR(diffusion, t):
    """
    Compute the signal-to-noise ratio for a single timestep.
    """
    diffusion.alpha_bar = diffusion.alpha_bar.to(t.device)
    return diffusion.alpha_bar[t] / (1 - diffusion.alpha_bar[t])

def caculate_loss(args, model, diffusion, x_0):
    """
    Importance sampling을 사용하기 위해 nn.MSE로 loss를 계산하지 않는다.
    Batch별 즉, user별 loss를 계산한 다음 importance sampling을 위해 Lt_history에 저장한다.
    그 다음, batch별 평균 값을 loss로 활용한다.
    """

    timesteps, pt = diffusion.sample_steps(x_0.shape[0])
    if args.noise_scale != 0:
        x_t, noise = diffusion.get_noised_interaction(x_0, timesteps)
    else:
        # Denoise 과정이 없다.
        # 즉, forward가 없다. 이러면 각 reverse는 x_t -> x_t를 복원하는 것이다.
        # 결국 x_t -> x_t를 하는 AutoEndocer를 여러 개 쌓은 것과 같다.
        x_t = x_0


    x_0_hat = model(x_t, timesteps)

    loss_batch_item = (x_0_hat - x_0) ** 2
    loss_batch = loss_batch_item.mean(dim=1)

    if args.snr is True:
        # timestep마다 loss weight를 다르게 둔다.
        weight = SNR(diffusion, timesteps - 1) - SNR(diffusion, timesteps)
        weight = torch.where((timesteps == 0), 1.0, weight)
    else:
        weight = torch.tensor([1.0] * x_0.shape[0]).to(args.device)
    
    weighted_loss_batch = weight * loss_batch

    # Update Lt_history & Lt_count for importance sampling
    for timestep, loss in zip(timesteps, weighted_loss_batch):
        # loss는 timestep에 해당하는 loss 값이다.
        if diffusion.Lt_count[timestep] == diffusion.num_for_expectation:
            # 만약 history가 꽉 찼으면 old한 것을 버리고 새 것으로 채운다.
            Lt_history_old = diffusion.Lt_history.clone()
            diffusion.Lt_history[timestep, :-1] = Lt_history_old[timestep, 1:]
            diffusion.Lt_history[timestep, -1] = loss.detach()
        else:
            try:
                diffusion.Lt_history[timestep, diffusion.Lt_count[timestep]] = loss.detach()
                diffusion.Lt_count[timestep] += 1
            except:
                print(timestep)
                print(diffusion.Lt_count[timestep])
                print(loss)
                raise ValueError
    
    weighted_loss_batch /= pt # 논문 수식 14 참고.
    weighted_loss = weighted_loss_batch.mean()
    
    return weighted_loss


def train_one_epoch(args, model, diffusion, optimizer, dataloader):
    batch_count = 0
    total_loss = 0.0
    for x_0 in dataloader:
        x_0 = x_0.to(args.device)
        batch_count += 1
        loss = caculate_loss(args, model, diffusion, x_0)

        total_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)


def compute_metric(target_items, predict_items, topK):
    precisions = []
    recalls = []
    ndcgs = []
    mrrs = []
    num_users = len(predict_items)

    for k in topK:
        sum_precision = sum_recall = sum_ndcg = sum_mrr = 0.0
        for user_id in range(num_users):
            if len(target_items[user_id]) == 0: continue
            mrr_flag = True
            num_hit = user_mrr = dcg = 0
            
            for rank_idx in range(k):
                if predict_items[user_id][rank_idx] in target_items[user_id]:
                    num_hit += 1 # precision, recall에 사용
                    dcg += 1.0 / np.log2(rank_idx + 2)                    
                    if mrr_flag:
                        user_mrr = 1.0 / (rank_idx+1.0)
                        mrr_flag = False
            
            idcg = 0.0
            for rank_idx in range(len(target_items[user_id])):
                idcg += 1.0/np.log2(rank_idx+2)
            ndcg = (dcg/idcg)

            sum_precision += num_hit / k
            sum_recall += num_hit / len(target_items[user_id])
            sum_ndcg += ndcg
            sum_mrr += user_mrr

        precision = round(sum_precision / num_users, 4)
        recall = round(sum_recall / num_users, 4)
        ndcg = round(sum_ndcg / num_users, 4)
        mrr = round(sum_mrr / num_users, 4)

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        mrrs.append(mrr)

    return precisions, recalls, ndcgs, mrrs


def evaluate(args, model, diffusion, loader, label_items: sp.csr_matrix, \
            consumed_items: sp.csr_matrix, topK: list):
    """
    Args
        args                    : hyper-parameters
        model                   : 학습된 model
        diffsuion               : Diffusion
        loader                  : Test data loader // no_shffule
        label_items             : Ground Truth, shape: (num_users, num_items) 중에서 target item에만 1
        consumed_items   : training data에서 사용된 이미 user가 선호도를 보인 items
        topK                    : top K list ex) [10, 20, 50]
    """
    model.eval()
    num_user = label_items.shape[0]
    user_idx_list = list(range(label_items.shape[0]))
    # target_items.shape[0] 대신 consumed_items.shape[0]도 ㄱㅊ

    predict_items = []
    target_items = []

    for user_id in range(num_user):
        # user_id에 해당하는, sp.csr_matrix로 저장되어 있는 user의 label item id를 list로 저장.
        # nonzero()하면 (row array, col array) 반환.
        # col array: np.ndarray의 idx 값이 item id임.
        target_items.append(label_items[user_id,:].nonzero()[1].tolist())

    with torch.no_grad():
        for batch_idx, x_0 in enumerate(loader):
            start_batch_user_id = batch_idx*args.batch_size
            end_batch_user_id = start_batch_user_id + len(x_0)
            batch_consumed_items = consumed_items[user_idx_list[start_batch_user_id:end_batch_user_id]]
            x_0 = x_0.to(args.device)
            prediction = diffusion.sample_new_interaction(model, x_0, steps=args.sampling_steps, sampling_noise=False)
            prediction[batch_consumed_items.nonzero()] = -np.inf

            _, indices = torch.topk(prediction, topK[-1]) # shape (x_0[1].shape, topK[-1])
            indices = indices.detach().cpu().numpy().tolist()
            predict_items.extend(indices)

        precisions, recalls, ndcgs, mrrs = compute_metric(target_items, predict_items, topK)
    
    return precisions, recalls, ndcgs, mrrs


def print_metric_results(topK, results):
    metric_list = ['Precision', 'Recall', 'nDCG', 'MRR']
    for idx, metric in enumerate(metric_list):
        str_result = ''
        for k_idx, k in enumerate(topK):
            str_metric = f'{metric}@{k}'
            str_result += f'    {str_metric:14s}: {results[idx][k_idx]:.4f}'
        print(str_result)

#### 3.2. main

In [7]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

dict_args = {}
args = dotdict(dict_args)


args.dataset_name = 'ml-1m'
args.file_name = 'ratings.dat'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.batch_size = 400
args.lr = 1e-4
args.weight_decay = 0.0
args.epochs = 1000
args.topK = [10, 20, 50, 100]

# model hyper
args.hidden_dims = [1000]
args.norm = True

# diffusion hyper
args.beta_start = 1e-4
args.beta_end = 0.02
args.noise_scale = 0.1
args.steps = 100
args.snr = True # assign different weight to different timestep or not
args.sampling_steps = 0



dir_path = os.path.join(os.getcwd(), args.dataset_name)
dataset = PreProcess(dir_path, args.file_name, use_cache=True)
train_dataset = DataDiffusion(torch.FloatTensor(dataset.sp_train.toarray()))
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, pin_memory=True, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)

diffusion = Diffusion()
encoder_dims = [dataset.num_items] + args.hidden_dims
decoder_dims = encoder_dims[::-1]
model = ReverseDiffusion(encoder_dims, decoder_dims).to(args.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

best_recall, best_epoch = -100, 0
best_test_result = None
print("Start training")
for epoch in range(1, args.epochs + 1):
    if epoch - best_epoch >= 20: # early stopping
        print('-'*18)
        print('Exiting from training early')
        break

    model.train()    
    start_time = time.time()
    avg_loss = train_one_epoch(args, model, diffusion, optimizer, train_loader)
    print(f'Epoch {epoch}  train loss {avg_loss:.4f}')

    if epoch % 5 == 0:
        val_results = evaluate(
            args, model, diffusion, test_loader, dataset.sp_val, dataset.sp_train, args.topK)
        test_results = evaluate(
            args, model, diffusion, test_loader, dataset.sp_test, dataset.sp_train, args.topK)
    
        val_recalls = val_results[1]
        if val_recalls[1] > best_recall:
            best_recall, best_epoch = val_recalls[1], epoch
            print('  Update Best')

        print('  Validation data')
        print_metric_results(args.topK, val_results)
        print('  Test data')
        print_metric_results(args.topK, test_results)
        

Start training
Epoch 1  train loss 42.5007
Epoch 2  train loss 37.5769
Epoch 3  train loss 36.4889
Epoch 4  train loss 33.4127
Epoch 5  train loss 35.7734
  Update Best
  Validation data
    Precision@10  : 0.0388    Precision@20  : 0.0352    Precision@50  : 0.0282    Precision@100 : 0.0223
    Recall@10     : 0.0486    Recall@20     : 0.0859    Recall@50     : 0.1667    Recall@100    : 0.2536
    nDCG@10       : 0.0429    nDCG@20       : 0.0604    nDCG@50       : 0.0908    nDCG@100      : 0.1183
    MRR@10        : 0.1006    MRR@20        : 0.1096    MRR@50        : 0.1155    MRR@100       : 0.1174
  Test data
    Precision@10  : 0.0636    Precision@20  : 0.0590    Precision@50  : 0.0466    Precision@100 : 0.0381
    Recall@10     : 0.0393    Recall@20     : 0.0754    Recall@50     : 0.1423    Recall@100    : 0.2250
    nDCG@10       : 0.0438    nDCG@20       : 0.0647    nDCG@50       : 0.0967    nDCG@100      : 0.1295
    MRR@10        : 0.1478    MRR@20        : 0.1588    MRR@50    